<a href="https://www.kaggle.com/code/baseershah/regression-with-polynomial-regression?scriptVersionId=252122958" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1>Flood Prediction with Regression</h1>


<h2>Import Libraries</h2>


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lgbm-poly3-robust/lgbm_poly_3_robust.txt
/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


<h2>Load Dataset</h2>

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

In [3]:
X = train.copy()

In [4]:
del train

In [5]:
X.drop('id',axis=1,inplace=True)

In [6]:
y=X.pop('FloodProbability')

<h2>Preprocessing</h2>

In [7]:
from sklearn.preprocessing import PolynomialFeatures
# poly_features = PolynomialFeatures(degree=2,include_bias=True)
poly_features_3 = PolynomialFeatures(degree=3, include_bias=True)

In [8]:
X['sum_of_features'] = X.sum(axis=1)

<h3>Downcasting for faster computation and lower memory usuage </h3>

In [9]:
X_float32 = X.astype('float32')

In [10]:
X_float32.shape

(1117957, 21)

<h3>Fitting polynomial with degree 3 in chunks</h3>

In [11]:
X_poly_3_1 = poly_features_3.fit_transform(X_float32[0:100000])
X_poly_3_2 = poly_features_3.fit_transform(X_float32[100000:200000])
X_poly_3_3 = poly_features_3.fit_transform(X_float32[200000:300000])
X_poly_3_4 = poly_features_3.fit_transform(X_float32[300000:400000])
X_poly_3_5 = poly_features_3.fit_transform(X_float32[400000:500000])
X_poly_3_6 = poly_features_3.fit_transform(X_float32[500000:600000])
X_poly_3_7 = poly_features_3.fit_transform(X_float32[600000:700000])
X_poly_3_8 = poly_features_3.fit_transform(X_float32[700000:800000])
X_poly_3_9 = poly_features_3.fit_transform(X_float32[800000:900000])
X_poly_3_10 = poly_features_3.fit_transform(X_float32[900000:1000000])
X_poly_3_11 = poly_features_3.fit_transform(X_float32[1000000:1110000])
X_poly_3_12 = poly_features_3.fit_transform(X_float32[1110000:1117958])


In [12]:
del X
del X_float32

In [13]:
X_poly_3_new_1 = pd.concat([pd.DataFrame(X_poly_3_1), pd.DataFrame(X_poly_3_2), pd.DataFrame(X_poly_3_3)],
                     ignore_index=True)

In [14]:
del X_poly_3_1
del X_poly_3_2
del X_poly_3_3


In [15]:
X_poly_3_new_2 = pd.concat([pd.DataFrame(X_poly_3_4),pd.DataFrame(X_poly_3_5), pd.DataFrame(X_poly_3_6)],
                     ignore_index=True)

In [16]:
del X_poly_3_4
del X_poly_3_5
del X_poly_3_6


In [17]:
X_poly_3_new_3 = pd.concat([pd.DataFrame(X_poly_3_7), pd.DataFrame(X_poly_3_8), pd.DataFrame(X_poly_3_9)],
                     ignore_index=True)

In [18]:
del X_poly_3_7
del X_poly_3_8
del X_poly_3_9


In [19]:
X_poly_3_new_4 = pd.concat([pd.DataFrame(X_poly_3_10), pd.DataFrame(X_poly_3_11), pd.DataFrame(X_poly_3_12)],
                     ignore_index=True)

In [20]:
del X_poly_3_10
del X_poly_3_11
del X_poly_3_12


In [21]:
import gc
gc.collect()

30

In [22]:
X_poly_3_new = pd.concat([pd.DataFrame(X_poly_3_new_1),pd.DataFrame(X_poly_3_new_2),pd.DataFrame(X_poly_3_new_3),pd.DataFrame(X_poly_3_new_4)],ignore_index=True)

In [23]:
X_poly_3_new = pd.DataFrame(X_poly_3_new)

In [24]:
del X_poly_3_new_1
del X_poly_3_new_2
del X_poly_3_new_3
del X_poly_3_new_4

In [25]:
import gc
gc.collect()

0

<h3>Robust Scaling the df</h3>

In [26]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_poly_3_scaled = scaler.fit_transform(X_poly_3_new)

In [27]:
X_poly_3_scaled = pd.DataFrame(X_poly_3_scaled)
X_poly_3_scaled

,0,1,2,3,4,5,6,7,8,9,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,0.0,0.000000,1.000000,0.0,1.5,0.333333,-0.333333,-0.5,-0.666667,-0.666667,...,1.153439,0.326389,0.816504,-0.256944,-0.075648,0.513295,-0.518519,-0.542569,-0.632007,-0.375860
1,0.0,0.333333,0.666667,-0.5,-0.5,1.000000,1.000000,-1.0,0.000000,-0.333333,...,-0.322751,-0.361111,-0.306189,-0.444444,-0.513990,-0.349400,-0.518519,-0.542569,-0.632007,-0.375860
2,0.0,0.333333,0.000000,0.5,1.0,-0.666667,0.666667,-2.0,0.000000,-0.333333,...,-0.518519,-0.506944,-0.528049,-0.506944,-0.636788,-0.542747,-0.518519,-0.526335,-0.538524,0.098909
3,0.0,-0.666667,-0.333333,0.5,0.0,-0.333333,1.000000,-0.5,0.666667,0.333333,...,1.153439,1.006944,0.993847,0.520833,0.787565,0.964364,0.000000,0.090188,0.258331,0.624140
4,0.0,0.000000,-0.666667,-1.5,0.5,-0.333333,-0.333333,-1.0,-0.666667,-0.666667,...,-0.518519,-0.381944,-0.615997,-0.173611,-0.538860,-0.993524,0.000000,-0.198413,-0.650995,-1.929944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,0.0,-0.666667,-0.666667,-0.5,2.5,-0.333333,0.000000,0.0,0.666667,1.666667,...,0.481481,0.305556,0.439377,-0.027778,0.132642,0.414163,-0.322751,-0.276335,-0.222036,0.098909
1117953,0.0,-1.000000,-1.000000,-0.5,-1.0,1.333333,0.000000,1.5,-1.333333,-0.666667,...,-0.322751,-0.138889,-0.294607,0.000000,-0.103627,-0.299932,0.000000,0.018038,0.000000,-0.191845
1117954,0.0,0.666667,-0.666667,2.0,-0.5,0.333333,0.000000,2.0,-1.333333,-0.666667,...,-0.619048,-0.583333,-0.708650,-0.472222,-0.692228,-0.874540,-0.322751,-0.282107,-0.247481,0.000000
1117955,0.0,0.666667,-0.666667,-1.0,1.0,0.000000,-1.000000,-1.0,-0.333333,0.333333,...,0.481481,0.305556,0.439377,-0.027778,0.132642,0.414163,-0.322751,-0.276335,-0.222036,0.098909


In [28]:
del X_poly_3_new

<h2>Lightgbm Training</h2>

<h3>Params defined</h3>

In [29]:
# lgb_params = {
#     'boosting_type': 'gbdt', 
#     'n_estimators':2000, 
#     'learning_rate' :  0.012, 
#     'device':'gpu',
#     'num_leaves' : 250, 
#     'subsample_for_bin': 165700, 
#     'min_child_samples': 114, 
#     'reg_alpha': 2.075e-06, 
#     'reg_lambda': 3.839e-07, 
#     'colsample_bytree': 0.9634,
#     'subsample': 0.9592, 
#     'max_depth': 10,
#     'random_state':0,
#     'verbosity':-1}

In [30]:
# lgb_reg = lgb.LGBMRegressor(**lgb_params)
# lgb_reg.fit(X_poly_3_scaled,y)

In [31]:
# lgb_reg.booster_.save_model('lgbm_poly_3_robust.txt')

<h3>saved model loaded back</h3>

In [32]:
booster = lgb.Booster(model_file='/kaggle/input/lgbm-poly3-robust/lgbm_poly_3_robust.txt')

<h2>Test Preprocessing</h2>

In [33]:
idx= test['id']
test.drop('id',axis=1,inplace=True)
test['sum_of_features'] = test.sum(axis=1)
test_float32 = test.astype('float32')
test_poly = poly_features_3.transform(test_float32)
test_poly_scaled = scaler.transform(test_poly)

In [34]:
del test
del test_float32
del test_poly

In [35]:
gc.collect()

9

<h2>Predictions + Submission</h2>

In [36]:
lgbm_preds = booster.predict(test_poly_scaled)

In [37]:
submission_df = pd.DataFrame({
    'id':idx,
    'FloodProbability':lgbm_preds
})

In [38]:
submission_df.to_csv('submission.csv',index=False)